<a href="https://colab.research.google.com/github/Ashonet/S-P100_ANALYSIS/blob/main/SP500_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def get_sp500_companies():
    sp500_symbols = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    sp500_symbols = sp500_symbols['Symbol'].tolist()
    return sp500_symbols

def get_intrinsic_value_dividend(symbol):
    try:
        data = yf.Ticker(symbol)
        dividends = data.dividends

        if len(dividends) < 2:
            return None  # Return only the intrinsic value if there's insufficient data

        growth_rate = (dividends.iloc[-1] - dividends.iloc[0]) / dividends.iloc[0]  # Calculate growth rate based on dividends

        latest_dividend = dividends.iloc[-1]
        discount_rate = 0.1  # Set your required rate of return

        # Ensure growth rate is not greater than the discount rate to avoid issues
        growth_rate = min(growth_rate, discount_rate * 0.9)  # Set a maximum growth rate

        intrinsic_value = latest_dividend / (discount_rate - growth_rate)
        return intrinsic_value

    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None

def get_intrinsic_value_relative(symbol):
    try:
        data = yf.Ticker(symbol)
        earnings_per_share = data.info.get('trailingEps', None)
        book_value_per_share = data.info.get('bookValue', None)

        if earnings_per_share is None or np.isnan(earnings_per_share) or book_value_per_share is None or np.isnan(book_value_per_share):
            return None  # Return None if financial data is insufficient or invalid

        # Relative Value
        intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)
        return intrinsic_value

    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None

def calculate_residual_income(book_value, cost_of_equity, future_residual_income):
    # Calculate intrinsic value using the residual income model
    intrinsic_value = book_value * (1 + (future_residual_income * 0.01) / (1 + cost_of_equity * 0.01))
    return intrinsic_value

def estimate_future_residual_income(symbol):
    try:
        data = yf.Ticker(symbol)
        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None

        # Fetch historical data (assuming quarterly data)
        historical_data = data.history(period="1y")  # Adjust period as needed

        if historical_data.empty or len(historical_data) < 4:  # At least 4 quarters needed for trend estimation
            return "Insufficient historical data for analysis", None

        # Calculate past residual incomes based on historical book values
        historical_book_values = historical_data['Close'].to_list()  # Assuming 'Close' prices represent book values
        past_residual_incomes = [0]  # Initialize with zero as the starting residual income

        for i in range(1, len(historical_book_values)):
            residual_income = max(0, historical_book_values[i] - book_value_per_share)
            past_residual_incomes.append(residual_income)

        # Estimate growth rate based on past performance (simple average growth rate)
        average_growth_rate = sum(past_residual_incomes[-4:]) / len(past_residual_incomes[-4:])

        # Project future residual income using the average growth rate
        future_residual_income = past_residual_incomes[-1] + average_growth_rate

        return future_residual_income
    except Exception as e:
        print(f"Error estimating future residual income for {symbol}: {e}")
        return None

def get_beta(symbol):
    try:
        stock_data = yf.Ticker(symbol)
        stock_history = stock_data.history(period='max')

        market_index = '^GSPC'  # S&P 500 index
        market_data = yf.Ticker(market_index).history(period='max')

        cov_stock_market = stock_history['Close'].pct_change().cov(market_data['Close'].pct_change())
        var_market = market_data['Close'].pct_change().var()

        beta = cov_stock_market / var_market
        return beta
    except Exception as e:
        print(f"Error fetching beta for {symbol}: {e}")
        return None

def calculate_cost_of_equity(symbol):
    try:
        risk_free_rate = 0.04  # Replace with actual risk-free rate (e.g., 10-year Treasury yield)
        market_return = 0.08  # Replace with expected market return

        beta = get_beta(symbol)
        if beta is None:
            return None

        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
        return cost_of_equity
    except Exception as e:
        print(f"Error calculating cost of equity for {symbol}: {e}")
        return None

def get_intrinsic_value_residual_income(symbol, cost_of_equity):
    try:
        data = yf.Ticker(symbol)
        historical_data = data.history(period="max")

        if historical_data.empty:
            return None  # Return None if there's insufficient data for analysis

        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return None  # Return None if financial data is insufficient or invalid for analysis

        # Calculate future residual income based on historical performance
        future_residual_income = estimate_future_residual_income(symbol)

        if future_residual_income is None:
            return None  # Return None if unable to estimate future residual income

        # Call the function to calculate intrinsic value using estimated future residual income
        intrinsic_value = calculate_residual_income(book_value_per_share, cost_of_equity, future_residual_income)
        return intrinsic_value

    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None


def calculate_average_intrinsic_value(symbol):
    intrinsic_values = []

    # Get intrinsic value from dividend approach
    intrinsic_value_dividend = get_intrinsic_value_dividend(symbol)
    if intrinsic_value_dividend is not None:
        intrinsic_values.append(intrinsic_value_dividend)

    # Get intrinsic value from relative approach
    intrinsic_value_relative = get_intrinsic_value_relative(symbol)
    if intrinsic_value_relative is not None:
        intrinsic_values.append(intrinsic_value_relative)

    # Get intrinsic value from residual income approach
    cost_of_equity = calculate_cost_of_equity(symbol)
    if cost_of_equity is not None:
        intrinsic_value_residual_income = get_intrinsic_value_residual_income(symbol, cost_of_equity)
        if intrinsic_value_residual_income is not None:
            intrinsic_values.append(intrinsic_value_residual_income)

    if len(intrinsic_values) > 0:
        average_intrinsic_value = np.mean(intrinsic_values)
        return average_intrinsic_value
    else:
        return None

def get_stock_rating(symbol, current_price):
    hold_range_percentage = 5
    intrinsic_value = calculate_average_intrinsic_value(symbol)

    if intrinsic_value is not None:
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    else:
        return None, None, None  # Return None if intrinsic value cannot be calculated

def main():
    # Get S&P 100 company symbols
    sp500_symbols = get_sp500_companies()

    # Example usage:
    start_date = '2019-12-01'
    end_date = '2023-12-20'

    for symbol in sp500_symbols:
        # Get historical data for the given symbol
        historical_data = get_stock_data(symbol, start_date, end_date)

        if historical_data.empty:
            print(f"Insufficient data for analysis for {symbol}")
        else:
            # Extracting the current price from historical data
            current_price = historical_data['Close'][-1]

            recommendation, intrinsic_value, price_type = get_stock_rating(symbol, current_price)

            if recommendation is not None and intrinsic_value is not None and price_type is not None:
                if recommendation == "Buy":
                    print(f"{symbol}: {recommendation} - {price_type} - {intrinsic_value:.2f}")
                elif recommendation == "Sell":
                    print(f"{symbol}: {recommendation} - {price_type} - {intrinsic_value:.2f}")
                elif recommendation == "Hold":
                    print(f"{symbol}: {recommendation} - {price_type} - {intrinsic_value[0]:.2f} to {intrinsic_value[1]:.2f}")
            else:
                print(f"No data available for {symbol}")

if __name__ == "__main__":
    main()
